# 前提条件

## ホストマシンに物理NICが3つあって、1つは上流に接続、2つはブリッジに繋がった状態であること

/etc/network/interfacesの例
```
auto lo
iface lo inet loopback

# 1つは上流に接続
auto eth0
iface eth0 inet dhcp

# 2つはブリッジに繋がった状態
auto enx14
iface enx14 inet manual

auto enx19
iface enx19 inet manual

auto br14
iface br14 inet static
address 192.168.14.1
netmask 255.255.255.0
bridge_ports enx14
bridge_maxwait 0
bridge_fd 0
bridge_stp off

auto br19
iface br19 inet static
address 192.168.19.1
netmask 255.255.255.0
bridge_ports enx19
bridge_maxwait 0
bridge_fd 0
bridge_stp off
```

## 同梱のdocker-compose.ymlでコンテナ一式が起動した状態であること

[同梱のdocker-compose.yml](/edit/wifi/docker-compose.yml)

# Juputerの設定

タイムゾーンをJSTにする。dockerコマンドをインストールする。

In [ ]:
%%bash
sudo -i
echo Asia/Tokyo > /etc/timezone
dpkg-reconfigure --frontend noninteractive tzdata
curl -fsSL https://download.docker.com/linux/static/stable/x86_64/docker-17.09.0-ce.tgz \
  | tar -xzC /usr/local/bin --strip=1 docker/docker
docker ps -a

# VyOSの設定

vyosには3つのネットワークを繋ぐが、docker-composeの挙動により、default以外のネットワークがコンテナにアタッチされる順番が保証されず、ネットワークとinterfaceの対応関係が固定されない（順序を固定する方法が見つからなかった）。そこで、wireのアタッチをコンテナ起動後に追加設定として行う。

因みに、docker createして、ネットワークをアタッチして、startさせる場合は、bridgeネットワークの名前順で認識される。dokcerの使われ方の主流では、デバイスとか気にしない感じで問題ないので、この辺の作りがイマイチなままなんだろうな。

In [ ]:
!sudo docker exec vyos ifconfig

In [ ]:
!sudo docker network ls

In [ ]:
!sudo docker network connect root_wire vyos

In [ ]:
!sudo docker exec vyos ifconfig

vyosのルータ設定はansible経由で行う。connection:dockerで接続するため、vyos_configが使えないので、スクリプトファイルをコピーしてシェル実行する。併せて、docker execからvyosのコマンドを叩くためのスクリプトファイルもコピーしておく。

[vyosの設定](/edit/wifi/vyos-setup.vsh)

[コマンドのラッパ](/edit/wifi/run-command.sh)

In [ ]:
!sudo ansible-playbook -i vyos, vyos-setup.yml

In [ ]:
!sudo docker exec --user vyos vyos /home/vyos/run-command.sh show interfaces

因みに、VyOSは実質Debianなので、interfaceが絡んだ機能（例えば、FireWallやQoS）は、OSのネットワーク設定で実現されている。このため、本事例のようにコンテナで利用すると、ネットワークデバイスが完全仮想化されていないことから、ネットワーク関連の機能のいくつかが動作しない。vyosにdocker execしてドヤ顔するのが不要で、そこそこのスペックのマシンが用意できるなら、普通にxenとか使った完全仮想化の方が、トラブルも少ないだろう。

# Aironetの設定

工場出荷時設定のAironet1140を2台、wifiのネットワークに繋ぐ。vyosの設定に、macアドレスに対して固定のIPアドレスを払い出すように書いてあるので、Aironetがそのアドレスを掴めたか確認する。起動にそこそこ時間がかかるので、時間を空けて何度か試すこと。

In [ ]:
!ping -c 4 192.168.14.64
!ping -c 4 192.168.14.83

工場出荷時設定のAironetは、telnetでデフォルトアカウントで入れるようになっているので、新規のアカウントで、sshでしか入れないように設定する。

[playbook](/edit/wifi/aironet-setup.yml)

In [ ]:
!ansible-playbook -i 192.168.14.64,192.168.14.83, aironet-setup.yml -t ssh-setup

## known_hostsにfingerprint 覚えさせる部分は手作業

ansibleがAironetにsshするためにknown_hostsにfingerprintを覚えさせる必要があります。ansibleの設定で自動登録できるらしいのですが、jupyter内に入れてあるansibleのバージョンの問題か、動作しなかったので、terminalからの手作業で対応します。playbook内のアカウントを確認して、[一覧ページ](/tree/wifi)の右上「New」-「Terminal」からterminalを開いて、ssh接続を行ってください。

```
$ ssh wifiuser@192.168.14.64
```

Aironetへのssh接続でfingerprintの確認が出ない状態にできたら、アクセスポイントの設定を行えます。

In [ ]:
!ansible-playbook -i 192.168.14.64,192.168.14.83, aironet-setup.yml -t setup

# 接続の確認

wifiで繋いだり、有線で繋いだりして、インターネットへの接続が行えることを確認します。